<figure>
  <IMG SRC="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Fachhochschule_Südwestfalen_20xx_logo.svg/320px-Fachhochschule_Südwestfalen_20xx_logo.svg.png" WIDTH=250 ALIGN="right">
</figure>

# Einführung Machine Learning
### Sommersemester 2022
Prof. Dr. Heiner Giefers

# Regressionsanalyse

Die Regressionsanalyse ist ein statistisches Verfahren, mit dem man eine beobachtete, abhängige Variable durch eine oder mehrere unabhängige Variablen zu erklären versucht.
Die **Lineare Regression** ist dabei ein Sonderfall der Regressionsanalyse, bei dem zur Beschreibung der Daten ein lineares Modell herangezogen wird. Das bedeutet, die abhängige Variable wird als Linearkombination der (mit den Regressionskoeffizienten) gewichteten unabhängigen Variablen beschrieben.
Im Gegensatz zur Klassifikation, werden bei der Regressionsanalyse immer Werte für einer kontinuierlichen Variable vorhergesagt.
Beispiele sind etwa Schätzer für folgende Fragestellungen:
- Was ist der angemessene Preis für ein Produkt
- Wie groß ist der Absatzmarkt für ein neues Produkt
- Wie lang ist die Halbarkeit eines Gerätes
- Wie hoch sind die Kosten eines bestimmten Projektes





## Daten aufbereiten und analysieren

Am Anfang aller Machine Learning Projekte stehen die Daten.
Ohne eine geeignete Datenbasis können Algorithmen keine neuen Einsichten oder Voraussagen treffen.

In diesem Modul werden wir hauptsächlich **strukturierte Daten** betrachten.
Strukturierte Daten orientieren sich nach einem vorgegebenen Format.
Eine Tabelle etwa ist ein strukturierter Datensatz, dann alle Werte sind einer bestimmten Kombination aus Zeile und Spalte im Datensatz zugeordnet.
Die einzelnen Spalten beschreiben dabei häufig *Kategorien* bzw. *Eigenschaften* der Daten.
Die Zeilen listen dann einzelnen Datenpunkte mit Werten für die entsprechenden Kategorien in den Spalten.

Unter **unstrukturierten Daten** versteht man Datensätze, die keine festgelegte Struktur besitzen.
Ein unstrukturierter Datensatz kann zwar eine Struktur in sich tragen, sie ist aber in dem vorliegenden Format nicht abgebildet und damit nicht direkt erkennbar.
Es gibt Machine Learning Algorithmen, die Strukturen in unstrukturierten Datensätzen aufspüren und sichtbar machen.
Zu den unstrukturierten Daten zählt man üblicherweise geschriebene und gesprochene Sprache (Texte und Sprache), Bilder und Videos.


In diesem Arbeitsblatt werden wir Daten betrachten, die von dem bekannten Online-Marktplatz für Gebrauchtfahrzeuge [AutoScout24](https://www.autoscout24.de) stammen.
Die Daten wurden am 11.05.2015 für ein Projekt an der ETH Zürich gesammelt.

Der Datensatz liegt als CSV (comma-separated values) Datei vor, einem gängigen Format für tabellarische Daten.
Wir benutzen die Python **Pandas** Bibliothek, um den Datensatz einzulesen und um ihn zu verarbeiten.
Die Funktion `read_csv` importiert eine CSV-Datei in ein `DataFrame` Objekt.
Da der Datensatz einige nicht-ASCII Zeichen enthält, verwenden wir das ISO-8859-1 (Latin-1) Zeichenformat beim Import.
Mit `head(10)` geben wir die ersten 10 Zeilen des Datensatzes aus.



In [ ]:
import pandas as pd
import os
import tarfile
import urllib.request

url = "https://github.com/fh-swf-hgi/ml/raw/main/u2/vw_golf_2015_complete.csv"
dfile = "./vw_golf_2015_complete.csv"

if not os.path.isfile(dfile):
    urllib.request.urlretrieve(url, dfile)
    
df = pd.read_csv("vw_golf_2015_complete.csv", encoding = "ISO-8859-1")
df.head(10)

Einige weitere Informationen, z.B. die Namen und Datentypen der Spalten, erhalten wir mit der Funktion `info()`:

In [ ]:
df.info()

Mit `describe()`erhalten wir einige Statistik-Informationen über die Spalten des Datensatzes:

In [ ]:
df.describe()

Die Datei enthält mit ca. 70.000 Datenpunkten recht viele Einträge.
Für unser Beispiel wollen wir den Datensatz etwas reduzieren.
Dazu wählen wir ein spezielles Automodell aus.
In der folgenden Zelle werden die Spalten *marke* und *modell* ausgewählt und in der resultierenden Tabelle alle doppelten Einträge verworfen. Wir erhalten so einen Dataframe der alle Modelltypen einmalig enthält.

In [ ]:
df_typen = df[["marke", "modell"]].drop_duplicates()

In [ ]:
df_typen.info()

Die Neue Tabelle enthält 705 unterschiedliche Typen.
Da die Indizes aus der ursprünglichen Tabelle übernommen werden, indizieren wir den neuen Datensatz neu durch.
Von dem resultierenden Dataframe geben wir die ersten 10 Zeilen aus.

In [ ]:
df_typen.reset_index(drop=True, inplace=True)
df_typen.loc[:9]

Mit `query("modell=='Golf VI'")` wählen wir die Golf-VI Modelle aus der Liste aus und speichern die Datenpunkte in einen neuen `DataFrame`.

Auch die Anzahl der Merkmale (Spalten) ist mit 54 recht hoch.
Daher reduzieren wir die Spalten des DataFrames auf die 10 Merkmale im Array `merkmale`.

In [ ]:
df_select = df.query("modell=='Golf VI'")

merkmale = ['klimaanlage','nichtraucherfahrzeug','anhaengerkupplung',
            'preis','erstzulassung_jahr','leistung','hubraum',
            'sitzplaetze','tueren','kilometer'
           ]

daten = df_select[merkmale]

In [ ]:
daten.info()

## Daten Visualisieren und mit Statistischen Kennwerten beschreiben

Als Nächstes möchten wir einen Eindruck erhalten, wie die einzelnen Merkmale miteinander zusammenhängen.
Ist die Anzahl der Merkmale überschaubar, bietet es sich an, **Streudiagramme** (engl. *scatter plot*) für alle Paare von Merkmalen zu zeichnen.

Streudiagramm werden die Elemente zweier Datenreihen paarweise (als X- und Y-Werte) in ein kartesisches Koordinatensystem eingetragen.
Dadurch ergibt sich eine sogenannte Punktwolke, die anzeigt, in welcher Abhängigkeit die beiden Datenreihen zueinander stehen.

Für Python gibt es verschiedene Bibliotheken, mit denen man Streudiagramme zeichnen kann.
Da unsere Daten als Pandas `DataFrame` vorliegen, bietet sich in unserem Fall die Funktion `scatter_matrix` aus dem Paket `pandas.plotting` an.
Da wir in ein kartesisches Koordinatensystem nur numerische Koordinaten eintragen können, reduzieren wir den DataFrame auf Spalten, die entweder als 64-bit `float` oder `int` Typen vorliegen.

In [ ]:
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
#%matplotlib notebook
%matplotlib --list

In [ ]:
%matplotlib inline

scatter_matrix(daten.select_dtypes(include=['float64','int64']), figsize=(10,10))

Die Abbildung zeigt nun eine Vielzahl von *Punktwolken*.
Uns interessiert hier vor allem die erste Zeile (oder die erste Spalte, da die Graphen an der Hauptdiagonalen gespiegelt sind).
Dort sind die Punktwolken abgebildet, bei denen die eine Datenreihe der Preis des Autos ist.
Wir wollen später eine **Schätzfunktion** trainieren, die möglichst gut den Preis eines Gebrauchtfahrzeugs vorhersagen kann.
Daher müssen wir herausfinden, mit welchen anderen Merkmalen der Preis möglichst eng linear *korreliert*.
Grafisch betrachtet, suchen wir nach Punktwolken, die sich möglichst eng anhand **einer** Geraden orientieren.

Erfreulicherweise sind wir nicht auf die visuelle Analyse der Daten angewiesen.
Der **Korrelationskoeffizient** $r$ ist eine Kenngröße, die angibt, wie stark die Werte zweier Datenreihen $x$ und $y$ miteinander linear korrelieren.
Der Wert von $r$ liegt immer in der Grenzen $[-1,1]$.
Dabei bedeutet ein sehr niedriger (etwa $r<-0.8$) oder ein sehr hoher Wert (etwa $r>0.8$), dass die Reihen korrelieren, es also einen statistischen Zusammenhang der Datenreihen gibt.
Ein Wert von $r$ um den Nullpunkt (etwa $-0.5<r<0.5$) bedeutet, dass es keinen signifikanten Zusammenhang gibt.

Für die Berechnung der **Korrelationsmatrix** mit der Pandas Funktion `corr()`, nehmen wir den Datensatz, der alle Merkmale enthält.
Aus der berechneten Matrix selektieren wir die Spalte *Preis* und geben die Korrelationskoeffizienten in absteigend sortierter Reihenfolge aus:

In [ ]:
corr_matrix = df_select.corr()
corr_matrix["preis"].sort_values(ascending=False)

Da für uns die absolut größten Werte interessant sind, wenden wir die Betragsfunktion `abs()` auf die Werte an und geben die sortierte Liste erneut aus.

In [ ]:
corr_matrix["preis"].abs().sort_values(ascending=False).head()

### Aufgabe: Berechnen Sie den Korrelationskoeffizienten $\rho$ für die Spalten *preis* und *kilometer* "per Hand"
Ziehen Sie dazu zuerst aus dem `DataFrame` `temp_df` mit dem Attribut `values` die Werte der (bereinigten) Datenreihen in NumPy Arrays.
Sie können zur Berechnung die NumPy Funktionen `mean()` (für den Mittelwert, bzw. Erwartungswert $\mu$) und `std()` (für die Standardabweichung $\sigma$) benutzen

$ \rho(X,Y) = \Large \frac{cov(X,Y)}{\sigma X\sigma Y}$  $\,$   mit  $\,$   $ \large cov(X,Y)=E\left[ (X-\mu X) (Y-\mu Y) \right]$

In [ ]:
import numpy as np


temp_df = df.query("modell=='Golf VI'")[["preis","kilometer"]].dropna()
ty = temp_df["preis"].values
tx = temp_df["kilometer"].values


In [ ]:
def cov_xy(x, y):
    ''' Berechne die Kovarianz cov(x,y)
        der beiden Merkmale x und y
    '''
    # YOUR CODE HERE
    raise NotImplementedError()
    
cov_xy(tx, ty)

In [ ]:
# Test Cell
#----------

test_x0 = np.array([  1.,  -4.,   7.,  -6., -14.,  -1.,  -3., -33.,   6.,   3.])
test_y0 = np.array([  2.,  -7., -14.,   3.,  -9.,  19.,  14.,  -3., -11., -23.])

assert cov_xy(test_x0, test_y0) == -18.46

In [ ]:
def calc_r(x, y):
    ''' Berechne den Korrelationskoeffizienten 
        roh(x, y) der beiden Merkmale x und y
    '''
    # YOUR CODE HERE
    raise NotImplementedError()

calc_r(tx, ty)

In [ ]:
# Test Cell
#----------

assert np.around(calc_r(test_x0, test_y0), 4) == -0.1362

NumPy stellt die Funktion `corrcoef()` zur Verfügung.
Damit können Sie ihr Ergebmis überprüfen:

In [ ]:
np.corrcoef(tx,ty)

## Daten selektieren und aufbereiten
Wir wollen uns für die weitere Analyse auf die Merkmale *erstzulassung_jahr*, *kilometer* und *leistung* konzentrieren, da sie die höchste Korrelation mit dem Merkmal *preis* aufweisen.

In [ ]:
golf_6_kompl = df.query("modell=='Golf VI'")
golf_6_num = golf_6_kompl[["preis","erstzulassung_jahr","kilometer","leistung"]]
golf_6_num.info()

Wenn wir den Datensatz auf diese Merkmale reduzieren, fällt auf, dass nicht alle Datenreihen die gleiche Anzahl an Werten beinhalten.
Diese "Lücken" im Datensatz sind kritisch für die weiteren Schritte.
Viele Algorithmen kommen mit unvollständigen Daten, bzw `NaN` Elementen nicht zurecht.
Daher eliminieren wir die Datenpunkte, die ein `NaN` enthalten mit der Funktion `dropna()` aus dem Datensatz.

In [ ]:
golf_6_num = golf_6_num.dropna()
golf_6_num.info()

Wenn wir für die ausgewählten Merkmale Streudiagramme plotten, sehen wir, dass sich für den Preis Abhängigkeiten der Art **"je mehr, desto mehr"** (*erstzulassung_jahr*, *leistung*) sowie **"je mehr, desto weniger"** (*kilometer*) abzeichnen.

In [ ]:
from pandas.plotting import scatter_matrix

scatter_matrix(golf_6_num, figsize=(10, 8))

Als nächstes wollen wir den Datensatz in 2 Teile aufteilen.
Einen **Trainingsdatensatz** sowie einen **Testdatensatz**.
Mit dem ersten *trainieren* wir unser Modell.
Mit dem zweiten Datensatz testen wir die trainierte Modellfunktion.
Mit diesem Ansatz adressieren wir das Problem des **Overfittings** (Überanpassung).
Es kann beim Trainieren des Modells dazu kommen, dass der Lern-Algorithmus die Modell-Parameter zu sehr auf den Trainingsdatensatz anpasst.
Durch vorhalten eines separaten Datensatzes, der nicht zum Trainieren benutzt wird, kann dieses Problem erkannt werden.

Es ist üblich, etwa 1/5 bis 1/3 der Daten für den Trainingsdatensatz und die restlichen Daten für den Testdatensatz zu verwenden.
Um die Daten aufzuteilen verwenden wir die Funktion `train_test_split` aus dem *sklearn* Modul `model_selection`.
Mit dem Parameter `test_size` bestimmen wir den Anteil der Testdaten. Mit `random_state` wird der Zufallszahlengenerator initialisiert, über den die Datenpunkte ausgewählt werden. Legt man `random_state` fest, so wird immer die gleiche Aufteilung vorgenommen. Dies hat den Vorteil, dass die Analysen vergleichbar sind.

In [ ]:
from sklearn.model_selection import train_test_split

y = golf_6_num['preis']
#X = golf_6_num.drop(['preis'],1)
# Wir wählen als einziges Merkmal die gefahrenen Kilometer aus
X = golf_6_num['kilometer']

# Da X nur ein Merkmal enthält, wird es von Pandas zur "Series" gemacht
# Wir benötigen aber einen DataFrame
X = pd.DataFrame(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Lineare Regression

Bisher haben wir ausschließlich den Datensatz grafisch und statistisch analysiert, aber noch kein Machine Learning Algorithmus angewendet.
Wir sind jetzt bereit, aus den gesammelten Daten eine *Schätzfunktion* abzuleiten.
Diese Funktion soll auf Grundlage eines oder mehrerer Merkmale eine Schätzung für den Preis des Fahrzeugs ausgeben.
Diese Schätzung soll so ausfallen, dass sie möglichst gut zu den bestehenden Datenpunkten passt.
damit könnte man z.B. der *Richtpreis* für eine neue Annonce berechnen oder bestimmen, ob ein Angebot im Vergleich eher teuer oder günstig ist.

Um diese Schätzfunktion zu entwickeln, stürzen wir uns auf die Zusammenhänge, die wir in der Analysephase herausgearbeitet haben und trainieren auf dieser Grundlage ein lineares Regressionsmodell.
Zu Beginn betrachten wir den einfachsten Fall, die sogenannte **Univariate Lineare Regression**.
Hierbei wird die abhängige Variable *preis* durch eine einzige unabhängige Variable, in diesem Fall den Kilometerstand, erklärt.
Die Modellfunktion lautet entsprechend, $h_{\Theta}(x)=\Theta_0+\Theta_1x$

Es ist durchaus möglich, das Regressionsmodell mit den Mitteln, die wir bisher kennengelernt haben (NumPy, Pandas), aufzustellen und zu trainieren.
Um die Trainingsalgorithmen kennenzulernen, werden wir das zu einem späteren Zeitpunkt auch tun.
Allerdings ist es in der Praxis eher unüblich, Machine Learning (ML) Modelle "per Hand" zu entwickeln.
Es gibt, insbesondere für Python, eine Vielzahl von Bibliotheken, mit denen ML Modelle effizient trainiert werden können.

Eine weit verbreitete ML Bibliothek für Python ist **Scikit-Learn** (oder auch *sklearn*).
Eine Besonderheit von Scikit-Learn ist, dass die Bibliothek sehr umfassend ist und ein breites Spektrum von ML-Algorithmen unterstützt.
Darunter sind Algorithmen zur Klassifikation, Regression und für das Clustering.
Auch unterstützende Methode wie etwa Algorithmen zur Dimensionsreduktion, zur Modellauswahl und zur Vorverarbeitung der Rohdaten sind in der Bibliothek enthalten.

Die Funktionen zur linearen Regression entnehmen wir aus dem Modul `sklearn.linear_model`.
Zur besseren Übersicht, weisen wir die abhängige Variable auf `y` (bzw. `y_test`) zu, die unabhängige Variable auf `X` (bzw. `X_test`)

Wir erzeugen mit `lin_reg` ein Modell als Objekt der Klasse `LinearRegression` und trainieren das Modell mit dem Aufruf `fit(X,y)` auf unsere Daten.

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

Nun haben wir das Modell trainiert, aber wie gut passt es zu unserem Datensatz?
Eine Antwort aus diese Frage kann uns der durchschnittlichen Fehler für unseren Trainingsdatensatz geben.
Die Funktion `mean_absolute_error` berechnet den Mittelwert der absoluten Fehler für die Schätzung `pred`.

In [ ]:
from sklearn.metrics import mean_absolute_error

avg_price = y_train.values.mean()
print("Mittlerer Preis: %d EUR" % avg_price)

pred = lin_reg.predict(X_train)
pred_err = mean_absolute_error(y_train,pred)
pred_err_pc = pred_err*100//avg_price

print("Mittlerer Fehler bei der Vorhersage: %d EUR (oder %d%%)" % (pred_err, pred_err_pc))

Allerdings haben wir hier den Trainingsdatensatz `(X_train,y_train)` benutzt.
Falls es beim Trainieren des Modells zu Overfitting-Problemen gekommen ist, würden wir diese so nicht erkennen.
Daher führen wir die Fehlerberechnung nun nochmals mit dem reservierten Testdatensatz aus:

In [ ]:
avg_price = y_test.values.mean()
print("Mittlerer Preis: %d EUR" % avg_price)

pred = lin_reg.predict(X_test)
pred_err = mean_absolute_error(y_test,pred)
pred_err_pc = pred_err*100//avg_price

print("Mittlerer Fehler bei der Vorhersage: %d EUR (oder %d%%)" % (pred_err, pred_err_pc))

Die gute Nachricht ist, dass unser Modell auch für neue Eingaben gleichartige Ergebnisse produziert.
Allerdings ist die Qualität der Schätzungen mit über 20%-igen Abweichung noch nicht optimal.
Das liegt vor allem daran, dass sich der Preis eines Gebrauchtwagens nicht nur an einem Merkmal festmachen lässt.
Daher werden wir im späteren Verlauf die Regression erneut, dann aber mit mehreren unabhängigen Variablen durchführen.

Zunächst bietet es sich aber an, die trainierte Funktion zu zeichnen.
Dazu benötigen wir die berechneten Parameter $\Theta$ der linearen Funktion $h_{\Theta}(x)=\Theta_0+\Theta_1x$.

Der *Niveauparameter* (oder *Achsenabschnitt*) $\Theta_0$ wird im Modell im Attribut `intercept_` abgelegt.
Alle weiteren Koeffizienten $\Theta_i$ finden sich im Array `coef_`.

In [ ]:
print("Theta_0 = ",  lin_reg.intercept_.item())
print("Theta_1 = ", lin_reg.coef_.item())

Mit diesen beiden Koeffizienten können wir die Modellfunktion $h$ aufstellen und über der Punktwolke des Datensatzes anzeigen.

In [ ]:
h = lin_reg.intercept_+lin_reg.coef_*X_test

fig, ax = plt.subplots()
ax.scatter(X_test,y_test)
ax.plot(X_test, h,'-r')
ax.set_xlabel("km Stand")
ax.set_ylabel("Preis")
plt.show()

**Aufgabe:** Verwenden Sie die gleiche Methode um die Abhängigkeit von `preis` und `leistung` zu untersuchen:

In [ ]:
from sklearn.linear_model import LinearRegression


# X: Unabhängige Variable
# y: Abhängige Variable

X = None
y = None

X_test = None
y_test = None

lin_reg = None

# Diese Parameter für train_test_split wählen
test_size=0.3
random_state=42

# YOUR CODE HERE
raise NotImplementedError()

(lin_reg.intercept_, lin_reg.coef_[0])

In [ ]:
# Hidden Test


Nun kann die Modellfunktion $h$ gebildet und geplottet werden

In [ ]:
h = lin_reg.intercept_+lin_reg.coef_*X_test

fig, ax = plt.subplots()
ax.scatter(X_test,y_test)
ax.plot(X_test, h,'-r')
ax.set_xlabel("leistung")
ax.set_ylabel("Preis")
plt.show()

## Multiple Lineare Regression

Nun wiederholen wir den Prozess und nehmen statt eine unabhängige Variable, die beiden Merkmale `kilometer`und `leistung` zum Modell hinzu.
Damit wird unser Modell zu einer **Multiplen Linearen Regression**.


In [ ]:
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split

y = golf_6_num['preis']
X = golf_6_num[['kilometer','leistung']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

                  
lin_reg2 = LinearRegression()
lin_reg2.fit(X_train, y_train)

avg_price = y_test.values.mean()
print("Mittlerer Preis: %d EUR" % avg_price)

pred = lin_reg2.predict(X_test)
pred_err = mean_absolute_error(y_test,pred)
pred_err_pc = pred_err*100//avg_price

print("Mittlerer Fehler bei der Vorhersage: %d EUR (oder %d%%)" % (pred_err, pred_err_pc))

Man sieht, dass sich die Schätzfunktion etwas verbessert hat.
Der mittlere Fehler ist von 21% auf 17% schrumpft.

Wir plotten nun auch die Modellfunktion für die Multiplen Lineare Regression mit zwei Parametern.
Wie wir sehen, spannt unsere Modellfunktion nun eine Ebene auf.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from time import time
%matplotlib notebook 

fig = plt.figure(figsize=(10,12))
ax = fig.add_subplot(111, projection='3d')

theta0 = lin_reg2.intercept_
theta1 = lin_reg2.coef_[0]
theta2 = lin_reg2.coef_[1]
(theta0, theta1, theta2)


# Erzeuge ein Gitter über die Fläche
n = 100
mn = np.min(X_test.values, axis=0)
mx = np.max(X_test.values, axis=0)

X,Y = np.linspace(mn[0], mx[0], n).reshape(1,-1), np.linspace(mn[1], mx[1], n).reshape(-1,1)
Z = theta0 + theta1*X + theta2*Y

ax.plot_surface(X, Y, Z, color='r', alpha=0.4)
ax.scatter(X_test["kilometer"], X_test["leistung"], y_test)
ax.view_init(20,800)
ax.set_xlabel("Kilometer")
ax.set_ylabel("Leistung")
ax.set_zlabel("Preis")
plt.show()


Nun nehmen wir mit dem Erstzulassungsjahr noch ein drittes Merkmal hinzu:

In [ ]:
from sklearn.linear_model import LinearRegression

y = golf_6_num['preis']
X = golf_6_num.drop('preis', 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
                  
lin_reg3 = LinearRegression(normalize=True)
lin_reg3.fit(X, y)

avg_price = y_test.values.mean()
print("Mittlerer Preis: %d EUR" % avg_price)

pred = lin_reg3.predict(X_test)
pred_err = mean_absolute_error(y_test,pred)
pred_err_pc = pred_err*100//avg_price

print("Mittlerer Fehler bei der Vorhersage: %d EUR (oder %d%%)" % (pred_err, pred_err_pc))

Erneut hat sich der mittlere Fehler verringert und mit 10% Fehlerquote gibt unsere Modellfunktion nun ganz passable Schätzungen.
Versuchen Sie, die Funktion weiter zu optimieren, etwa indem Sie weitere Parameter hinzunehmen oder geschickt  neue Merkmale aus bestehenden Merkmalen berechnen.